## Import des bibliothèques

In [1]:
pip install googleapis-common-protos protobuf grpcio pandas systemathics.apis statsmodels matplotlib seaborn scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import re
import grpc
import pandas as pd
import seaborn
import statsmodels.api as sm
import google.protobuf as pb
import systemathics.apis.services.static_data.v1.static_data_pb2 as static_data
import systemathics.apis.services.static_data.v1.static_data_pb2_grpc as static_data_service
import numpy as np
from statsmodels.tsa.stattools import adfuller,coint
import matplotlib.pyplot as plt
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.daily.v1.daily_prices_pb2 as daily_prices
import systemathics.apis.services.daily.v1.daily_prices_pb2_grpc as daily_prices_service
from datetime import datetime
import itertools
import copy
from scipy.optimize import minimize

In [3]:
## Authentification

token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

'Bearer eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImpwZDhjS2Z5Zi13QXkzOURpNENqWSJ9.eyJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9lbWFpbGFkZHJlc3MiOiJhcnRodXIudmlub3RAZWR1LmRldmluY2kuZnIiLCJpc3MiOiJodHRwczovL2dhbnltZWRlLXByb2QuZXUuYXV0aDAuY29tLyIsInN1YiI6ImF1dGgwfDYxNmQ4NzI5NWQzZDlkMDA3MGVkYmIxNCIsImF1ZCI6WyJodHRwczovL3Byb2QuZ2FueW1lZGUtcHJvZCIsImh0dHBzOi8vZ2FueW1lZGUtcHJvZC5ldS5hdXRoMC5jb20vdXNlcmluZm8iXSwiaWF0IjoxNjQ3Nzc0MzYyLCJleHAiOjE2NTAzNjYzNjIsImF6cCI6Ijl5R0tzbGtFczFWNm9xRk9aa0h0a1V0NWkyNTVackpJIiwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBlbWFpbCIsInBlcm1pc3Npb25zIjpbImFkbWluIiwiYXJ0ZWZhY3RzIiwiY2hhbmdlcyIsImNvbXBvbmVudHMiLCJkYWlseV9iYXJzIiwiZGFpbHlfYm9sbGluZ2VyIiwiZGFpbHlfY21hIiwiZGFpbHlfZW1hIiwiZGFpbHlfbWFjZCIsImRhaWx5X3ByaWNlcyIsImRhaWx5X3JzaSIsImRhaWx5X3NtYSIsImRhaWx5X3ZvbGF0aWxpdHkiLCJkYWlseV92d2FwcyIsImRpdmlkZW5kcyIsImlkZW50aWZpZXJfbWFwcGluZyIsImludHJhZGF5X2JhcnMiLCJpbnRyYWRheV9ib2xsaW5nZXIiLCJpbnRyYWRheV9jbWEiLCJpbnRyYWRheV9lbWEiLCJpbnRyYWRheV9tYWNkIiw

# Sélection des paires

## Choix des indicateurs de sélection

### Correlation

In [4]:
def correlation(timeseries1,timeseries2):
    return np.corrcoef(sample1_sum,sample2_sum)[0,1]
# Correlation need to be near 1

### Stationarity

In [5]:
def stationarity_test_bool(timeseries,cutoff=0.01):
    # H_0 in adfuller is unit root exists (non-stationary)
    # We must observe significant p-value to convince ourselves that the series is stationary
    pvalue = adfuller(timeseries, maxlag=3)[1]
    return True if pvalue<cutoff else False

In [6]:
def stationarity_test_pvalue(timeseries,cutoff=0.01):
    # H_0 in adfuller is unit root exists (non-stationary)
    # We must observe significant p-value to convince ourselves that the series is stationary
    return adfuller(timeseries, maxlag=3)[1]

### Cointegration

In [7]:
def cointegration_test(timeseries1,timeseries2):
    return coint(timeseries1,timeseries2, maxlag=3)[1]    # return the p-value of the test

# Low pvalue means high cointegration!

### Standardized data

In [8]:
def mean_norm(df_input):
    df_input.loc[:, df_input.columns != "Date"] = df_input.loc[:, df_input.columns != "Date"].apply(lambda x: (x-x.mean())/ x.std(), axis=0)
    return df_input
#We have to choose the best way to standardized the dataframe here

## Application des indicateurs de sélection

### Recueil des données

#### Recueil des tickers

In [9]:
# define a method to handle the equities reponse using a Pandas dataframe
def get_equities_dataframe(response):
    identifier = ['{0}|{1}'.format(equity.identifier.ticker, equity.identifier.exchange) for equity in response.equities]
    type = [equity.type for equity in response.equities]
    country = [equity.country for equity in response.equities]
    name = [equity.name for equity in response.equities]
    currency = [equity.currency for equity in response.equities]
    primary = [equity.primary for equity in response.equities]
    tick_size_rule = [equity.tick_size_rule for equity in response.equities]
    mapping = [get_mapping(equity.mapping) for equity in response.equities]
    index = [equity.index for equity in response.equities]
    open = [equity.open for equity in response.equities]
    close = [equity.close for equity in response.equities]
    time_zone = [equity.time_zone for equity in response.equities]
    lot_size = [equity.lot_size for equity in response.equities]
    point_value = [equity.point_value for equity in response.equities]
    isin = [equity.isin for equity in response.equities]
    cusip = [equity.cusip for equity in response.equities]
    sedol = [equity.sedol for equity in response.equities]
    sectors = [get_sectors(equity.sectors) for equity in response.equities]
    capitalization = [equity.capitalization.value for equity in response.equities]
    
    # Create pandas dataframe
    d = {'Identifier': identifier, 'Type': type, 'Country': country, 'Name': name, 'Currency': currency, 'Primary': primary, 'TickSizeRule': tick_size_rule, 'Mapping':mapping, 'Index': index, 'Open': open, 'Close': close, 'Time zone': time_zone, 'Lot size': lot_size, 'PointValue': point_value, 'Isin': isin, 'Cusip': cusip, 'Sedol': sedol, 'Sectors': sectors, 'Capitalization': capitalization}
    df = pd.DataFrame(data=d)
    return df

In [10]:
# define methods to handle identifiers mapping and sectors display as a string
def get_mapping(d):
    res=''
    for key, value in d.items():
        res = res + '['+key+'='+value+']'
    return res

def get_sectors(d):
    res=''
    for key, value in d.items():
        res = res + '['+key+','+value+']'
    return res

def get_identifier(d):
    res=''
    for key, value in d.items():
        res = res + '['+key+'='+value+']'
    return res

In [11]:
# generate static data request
request = static_data.StaticDataRequest( 
    asset_type = static_data.AssetType.ASSET_TYPE_EQUITY
)

#request.index.value = 'Nasdaq 100'
request.exchange.value = 'XNGS'     # Requête qui ne filtre que la bourse primaire mais pas la bourse réelle
request.count.value = 1000

In [12]:
# open a gRPC channel
with open(os.environ['SSL_CERT_FILE'], 'rb') as f:
    credentials = grpc.ssl_channel_credentials(f.read())
with grpc.secure_channel(os.environ['GRPC_APIS'], credentials) as channel:
    
    # instantiate the static data service
    service = static_data_service.StaticDataServiceStub(channel)
    
    # process the request
    response = service.StaticData(request = request, metadata = [('authorization', token)])

# visualize request results
data = get_equities_dataframe(response)

data = data[data.Identifier.str.split('|').str[1] == "XNGS"]        # Ces deux lignes réctifient le problème du filtre de la bourse 
data = data[data.Index.str.contains(r"Nasdaq 100") == True]          # dans la requete qui ne filtre pas complètement

data.sort_values('Identifier')

,Identifier,Type,Country,Name,Currency,Primary,TickSizeRule,Mapping,Index,Open,Close,Time zone,Lot size,PointValue,Isin,Cusip,Sedol,Sectors,Capitalization
50,AAPL|XNGS,Equity,US,Apple Inc Common Stock,USD,XNGS,[0:0.0001][1:0.01],[Idc|564=564|AAPL][Figi=BBG000B9XRY4][Bloomber...,Composite|Industrials|Nasdaq 100|Nasdaq Compos...,,,,1,1.0,US0378331005,037833100,2046251,"[Nasdaq,Computer Manufacturing][SIC,3571 Elect...",2.784724e+12
130,ABNB|XNGS,Equity,US,Airbnb Inc Cl A,USD,XNGS,[0:0.0001][1:0.01],[Figic=BBG001Y2XS07][Bloomberg=ABNB US Equity]...,Nasdaq 100|Nasdaq Composite|nasdaq100,,,,1,1.0,US0090661010,009066101,BMGYYH4,"[SIC,7340 Services-To Dwellings & Other Buildi...",9.396266e+10
104,ADBE|XNGS,Equity,US,Adobe Inc,USD,XNGS,[0:0.0001][1:0.01],[Figic=BBG000BB5006][Esignal=ADBE][Idc|564=564...,Nasdaq 100|Nasdaq Composite|Russell 1000|Russe...,,,,1,1.0,US00724F1012,00724F101,2008154,"[Nasdaq,Computer Software: Prepackaged Softwar...",2.098938e+11
2,ADI|XNGS,Equity,US,Analog Devices Inc,USD,XNGS,[0:0.0001][1:0.01],[Reuters=ADI.OQ][Figi=BBG000BB6G37][Esignal=AD...,Nasdaq 100|Nasdaq Composite|Russell 1000|Russe...,,,,1,1.0,US0326541051,032654105,2032067,"[SIC,3674 Semiconductors & Related Devices][Na...",8.371995e+10
61,ADP|XNGS,Equity,US,Automatic Data Processing Inc,USD,XNGS,[0:0.0001][1:0.01],[Reuters=ADP.OQ][Bloomberg=ADP US Equity][Figi...,Nasdaq 100|Nasdaq Composite|Russell 1000|Russe...,,,,1,1.0,US0530151036,053015103,2065308,"[Nasdaq,Business Services][SIC,7374 Services-C...",8.985608e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,WBA|XNGS,Equity,US,Walgreens Boots Alliance Inc,USD,XNGS,[0:0.0001][1:0.01],[Figic=BBG000BWLMJ4][Bloomberg=WBA US Equity][...,Composite|Industrials|Nasdaq 100|Nasdaq Compos...,,,,1,1.0,US9314271084,931427108,BTN1Y44,"[SIC,5912 Retail-Drug Stores and Proprietary S...",4.098816e+10
59,WDAY|XNGS,Equity,US,Workday Inc,USD,XNGS,[0:0.0001][1:0.01],[Figi=BBG000VC0T95][Figic=BBG000VC0T95][Reuter...,Nasdaq 100|Nasdaq Composite|Russell 1000|Russe...,,,,1,1.0,US98138H1014,98138H101,B8K6ZD1,"[SIC,7374 Services-Computer Processing & Data ...",5.906030e+10
95,XEL|XNGS,Equity,US,Xcel Energy Inc,USD,XNGS,[0:0.0001][1:0.01],[Figi=BBG000BCTQ65][Reuters=XEL.OQ][Esignal=XE...,Composite|Nasdaq 100|Nasdaq Composite|Russell ...,,,,1,1.0,US98389B1008,98389B100,2614807,"[SIC,4931 Electric & Other Services Combined][...",3.793170e+10
131,ZM|XNGS,Equity,US,Zoom Video Communications Cl A,USD,XNGS,[0:0.0001][1:0.01],[Reuters=ZM.OQ][Esignal=ZM][Bloomberg=ZM US Eq...,Nasdaq 100|Nasdaq Composite|Russell 1000|Russe...,,,,1,1.0,US98980L1017,98980L101,BGSP7M9,"[SIC,7370 Services-Computer Programming, Data ...",3.328951e+10


### Get Ticker List

In [13]:
def get_ticker_list(data):
    return data.Identifier.str.split('|').str[0].tolist()

In [14]:
ticker_list = get_ticker_list(data)

### Recupération des SIC (secteur)

In [15]:
def get_sic(data):      # Fonction qui permet de récupérer le code SIC d'un tableau d'equities contenant la colone ['Sectors']
    sic = []            # Cette fonction ajoute au dataframe une nouvelle colone appelée ['SIC'] au dataframe d'equities
    for i in range(len(data)):
        ligne = data.iloc[i]['Sectors']
        match = re.search(r"SIC,([0-9]{2})", ligne)
        sic.append(match.group().split(",")[1])
    data['SIC'] = sic
    return data

In [16]:
data = get_sic(data)

### Liste de Dataframes par SIC

In [17]:
def sep_secteur(data):
    groups = data.groupby(['SIC'])
    liste_sic = data['SIC'].unique()
    liste_sic.sort()
    df_SIC = []
    for i in range(len(liste_sic)):
        df_SIC.append(groups.get_group(liste_sic[i]))
    return df_SIC

In [18]:
data_sec = sep_secteur(data)   # Liste de DF avec entreprises par secteur (selon le SIC)

In [19]:
data_sec[0]

,Identifier,Type,Country,Name,Currency,Primary,TickSizeRule,Mapping,Index,Open,Close,Time zone,Lot size,PointValue,Isin,Cusip,Sedol,Sectors,Capitalization,SIC
3,MDLZ|XNGS,Equity,US,Mondelez International Ord Shs Class A,USD,XNGS,[0:0.0001][1:0.01],[Reuters=MDLZ.OQ][Bloomberg=MDLZ US Equity][Es...,Nasdaq 100|Nasdaq Composite|Russell 1000|Russe...,,,,1,1.0,US6092071058,609207105,B8CKK03,"[SIC,2000 Food and Kindred Products][Nasdaq,Me...",8.645119e+10,20
23,KHC|XNGS,Equity,US,Kraft Heinz Company,USD,XNGS,[0:0.0001][1:0.01],[Esignal=KHC][Figi=BBG005CPNTQ2][Idc|564=564|K...,Nasdaq 100|Nasdaq Composite|Russell 1000|Russe...,,,,1,1.0,US5007541064,500754106,BYRY499,"[SIC,2030 Canned, Frozen & Preserved Fruit, Ve...",4.619619e+10,20
31,MNST|XNGS,Equity,US,Monster Beverage Cp,USD,XNGS,[0:0.0001][1:0.01],[Esignal=MNST][Figi=BBG008NVB1C0][Bloomberg=MN...,Nasdaq 100|Nasdaq Composite|Russell 1000|Russe...,,,,1,1.0,US61174X1090,61174X109,BZ07BW4,"[Nasdaq,Beverages (Production/Distribution)][S...",4.143821e+10,20
40,KDP|XNGS,Equity,US,Keurig Dr Pepper Inc,USD,XNGS,[0:0.0001][1:0.01],[Figi=BBG000TJM7F0][Bloomberg=KDP US Equity][F...,Nasdaq 100|Nasdaq Composite|Russell 1000|Russe...,,,,1,1.0,US49271V1008,49271V100,BD3W133,"[Nasdaq,Beverages (Production/Distribution)][S...",5.360639e+10,20
62,PEP|XNGS,Equity,US,Pepsico Inc,USD,XNGS,[0:0.0001][1:0.01],[Idc|564=564|PEP][Figi=BBG000DH7JK6][Figic=BBG...,Nasdaq 100|Nasdaq Composite|Russell 1000|Russe...,,,,1,1.0,US7134481081,713448108,2681511,"[Nasdaq,Beverages (Production/Distribution)][S...",2.226527e+11,20


In [20]:
sic_list = [get_ticker_list(groupe) for groupe in data_sec]

In [21]:
sic_list[0]

['MDLZ', 'KHC', 'MNST', 'KDP', 'PEP']

## Création de toutes les paires possibles

#### Récupération des combinaisons dans chaque secteur

In [22]:
def list_combinations(liste_paires):   # On fait une combinaison de toutes les facons possibles de faire des paires
    combi = []
    for i in range(len(liste_paires)):
        combi.append(list(itertools.combinations(liste_paires[i],2)))
    return [x for sublist in combi for x in sublist if x != []]

In [23]:
combinations = list_combinations(sic_list)  # Listes de tuples avec toutes les combinaisons possibles

### Requete de prix pour chaque paire possible

#### Fonction qui va requêter les prix à travers l'API

In [24]:
def price_request(ticker, exchange):
    request = daily_prices.DailyPricesRequest( identifier = identifier.Identifier(exchange = exchange, ticker = ticker))
    
    # open a gRPC channel
    with open(os.environ['SSL_CERT_FILE'], 'rb') as f:
        credentials = grpc.ssl_channel_credentials(f.read())
    with grpc.secure_channel(os.environ['GRPC_APIS'], credentials) as channel:

        # instantiate the daily prices service
        service = daily_prices_service.DailyPricesServiceStub(channel)

        # process the daily prices request
        return service.DailyPrices(request = request, metadata = [('authorization', token)])

#### Cache

In [25]:
%%time
cache = pd.DataFrame({'Name' : ticker_list, 'Data' : [price_request(x, 'XNGS') for x in ticker_list]})

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "Internal Error in Daily Price Api"
	debug_error_string = "{"created":"@1647775210.055271438","description":"Error received from peer ipv4:10.45.196.79:5001","file":"src/core/lib/surface/call.cc","file_line":903,"grpc_message":"Internal Error in Daily Price Api","grpc_status":13}"
>

In [26]:
def df_paire(paire, exchange, cache):
    response1 = cache.iloc[int(cache.index[cache['Name'] == paire[0]].values), 1]
    response2 = cache.iloc[int(cache.index[cache['Name'] == paire[1]].values), 1]
    
    # prepare the dataframe content
    dates1 = [datetime(p.date.year, p.date.month, p.date.day) for p in response1.data]
    dates2 = [datetime(p.date.year, p.date.month, p.date.day) for p in response2.data]
    if (len(dates1) <= len(dates2)):
        dates = dates1
    else:
        dates = dates2
    prices1 = [p.price for p in response1.data][-len(dates):]       # So all arrays have the same length
    prices2 = [p.price for p in response2.data][-len(dates):]

    d = {'Date': dates, f'{paire[0]}': prices1, f'{paire[1]}': prices2}
    return pd.DataFrame(data=d)

#### Fonction qui agrège les requêtes entre elle pour classer les paires suivant leur groupe

In [27]:
def pair_combination(combinations, exchange, cache):
    temp = [df_paire(x, exchange, cache) for x in combinations]
    return [x for x in temp if len(x) > 100]

In [28]:
%%time
df_combi = pair_combination(combinations, 'XNGS', cache)  # Représente une liste avec les paires

NameError: name 'cache' is not defined

## Séparation entre les données de test et celles d'entrainement

#### Data Splitting

In [29]:
def addMonths(date):
    b = pd.Timestamp(date) + pd.DateOffset(months=1)
    return b

In [30]:
def sep_tableau_train_test(df, train_start_date = np.datetime64('2014-01-01'), train_end_date = np.datetime64('2019-01-01'), test_end_date = np.datetime64('2021-01-01')):    # Séparation entre les données de test et de train
    test_start_date = train_end_date
    
    test = []
    train = []
    for i in range(len(df)):
        if (df[i].Date.iloc[0] <= train_start_date):
            train_date = (df[i]['Date'] >= train_start_date) & (df[i]['Date'] < train_end_date)
            test_date = (df[i]['Date'] >= test_start_date) & (df[i]['Date'] <= test_end_date)
            train.append(df[i].loc[train_date])
            test.append(df[i].loc[test_date])
            
    return train, test

In [31]:
train, test = sep_tableau_train_test(df_combi)

NameError: name 'df_combi' is not defined

In [ ]:
### Calcul du nombre de Paires crées

"Nombre de paires = " + str(len(train))   # Nombre de paires

In [ ]:
list(train[0].columns)[:3]

## Fonctions de Sélections des paires

#### Calcul de la cointégration

In [ ]:
def coint_df(df):
    df['Cointegration'] = pd.Series(cointegration_test(df.iloc[:,1], df.iloc[:,2]), index = df.index[[0]])
    return df

In [ ]:
def add_coint(df_combi):
    return [coint_df(x) for x in copy.deepcopy(df_combi)]

#### Calcul de la stationnarité

In [ ]:
def statio_df(df, column_name, indicateur):
    df[column_name] = pd.Series(stationarity_test_pvalue(df[indicateur]), index = df.index[[0]])
    return df

In [ ]:
def add_statio(df, column_name, indicateur):
    return [statio_df(x, column_name, indicateur) for x in copy.deepcopy(df)]

#### Calcul du ratio

In [ ]:
def ratio_df(df):
    df['Ratio'] = df.iloc[:,1] / df.iloc[:,2]
    return df

In [ ]:
def add_ratio_list_df(df):
    return add_statio(df, 'Statio_Ratio', 'Ratio')

#### Calcul du log-ratio

In [ ]:
def log_ratio_df(df):
    df['Log_Ratio'] = np.log(df.iloc[:,1] / df.iloc[:,2])
    return df

In [ ]:
def add_log_list_df(df):
    return add_statio(df, 'Statio_Log_Ratio', 'Log_Ratio')

##### Ajout des ratios au df

In [ ]:
def add_ratios(df):
    df_copy = [ratio_df(x) for x in copy.deepcopy(df)]
    return [log_ratio_df(x) for x in df_copy]

#### Fonction qui enlève les paires peu cointégrées

In [ ]:
def drop_faible_coint(train, threshold):    # Retourne une liste de paires suffisament cointégrés
    return [x for x in train if (x['Cointegration'].iloc[0] < threshold)]

#### Fonction qui enlève les paires dont les 2 indicateurs sont non-stationnaires

In [ ]:
def drop_faible_statio(df_combi, threshold):    # Retourne une liste de paires suffisament cointégrés
    df_combi_copy = []
    count_ratio = 0
    count_log = 0
    same = 0

    for x in df_combi:
        if x['Statio_Ratio'].iloc[0] < threshold:     # On compte le nombre de stationnarité acceptable entre le ratio classique et le log_ratio
            count_ratio += 1
        if x['Statio_Log_Ratio'].iloc[0] < threshold:
            count_log += 1
        if x['Statio_Log_Ratio'].iloc[0] < threshold and x['Statio_Ratio'].iloc[0] < threshold:
            same += 1
            
    return [x for x in df_combi if (x['Statio_Log_Ratio'].iloc[0] < threshold)], count_ratio, count_log, same

#### Cointégration

In [ ]:
%%time
df_coint = add_coint(train)

### Fonction qui agrège les différentes fonctions de sélection (Log-Ratio, Ratio, Stationnarité des indicateurs) et qui enlève les paires qui ne valident pas les critères.

In [ ]:
def pair_selection(df_coint):
    df = [x for x in df_coint if (x['Cointegration'].iloc[0] < 0.05)]
    print("drop_faible_coint OK")
    df = add_ratios(df)
    print("Add_Ratios OK")
    df = add_log_list_df(df)
    print("Add_Log_List OK")
    df = add_ratio_list_df(df)
    print("Add_Ratio_List OK")
    return drop_faible_statio(df, 0.05)

In [ ]:
def test_selection(train, test):
    train_list = [list(x.columns)[:3] for x in train]
    return [x for x in test if list(x.columns) in train_list]

In [ ]:
def test_selection_full(train, test):
    return [x for x in test]

In [ ]:
%%time
train, ratio_score, log_score, same = pair_selection(df_coint)

test = test_selection(train, test)

In [ ]:
# test[0]
# w = (test[2]['Date'] >= np.datetime64('2019-01-02')) & (test[2]['Date'] < np.datetime64('2019-02-01'))

# test[2]['Selected'] = w
# test[2]

In [ ]:
test[2]

In [ ]:
train[0]

## Score de la sélection des paires

In [ ]:
"Score ratio = " + str(ratio_score) + " Score Log_Ratio = " + str(log_score) + " Same = " + str(same)

In [ ]:
#We plot a random cointegrate pair to verify if everything is ok
plt.figure(figsize=(25, 10))
plt.plot('Date', test[0].columns[1], data=test[0], marker='', color='blue', linewidth=1, alpha = 0.6)
plt.plot('Date', test[0].columns[2], data=test[0], marker='', color='red', linewidth=1, alpha = 0.6)
plt.ylabel('Price')
plt.xlabel('Date')
plt.title("{} and {} prices".format(test[0].columns[1],test[0].columns[2]))
plt.show()

In [ ]:
test[0].

# Investissement

In [ ]:
def calcul_moyenne_mobile(liste_df):
    window = 50
    liste_df = [log_ratio_df(df) for df in liste_df]
    liste_df = [ratio_df(df) for df in liste_df]
    
    for i in range(len(liste_df)):
        # Calcul de la moyenne mobile
        rolling_window = liste_df[i]['Log_Ratio'].rolling(window=window, center=False)
        rolling_window_mean = rolling_window.mean()
        rolling_window_std  = rolling_window.std()

        Z_score_rolling = (liste_df[i]['Log_Ratio'] - rolling_window_mean)/ rolling_window_std
        liste_df[i]['Zscore_rolling'] = Z_score_rolling
    return liste_df

In [ ]:
test = calcul_moyenne_mobile(test)
train = calcul_moyenne_mobile(train)

In [ ]:
#We plot a random cointegrate pair to verify if everything is ok
plt.figure(figsize=(25, 10))
plt.plot('Date','Log_Ratio', data=train[0][0])
plt.plot('Date','Log_Ratio', data=test[0][0])
plt.axhline(train[0][0].Log_Ratio.mean())
plt.ylabel('Log_Ratio')
plt.xlabel('Date')
plt.title("{} and {} Log_Ratio".format(test[0][0].columns[1],test[0][0].columns[2]))
plt.show()

In [ ]:
def graph_invest(df):
    # Band
    # Z-Score
    df = copy.deepcopy(df).iloc[:500] 
    plt.figure(figsize=(25, 10))
    plt.plot('Date', 'Zscore_rolling', data=df, marker='', color='blue', linewidth=1, alpha =  0.6, label='Zscore_rolling')
    day_count = 0
    
    position = False 
    for i in range(len(df)):
        day_count += 1
        if abs(df.Zscore_rolling.iloc[i]) > 2.5 and position == True: # Stop loss
            plt.plot(df.Date.iloc[i], df.Zscore_rolling.iloc[i], color='red', linestyle='None', marker='o')
            position = False
            day_count = 0
        
        elif day_count > 50:
            plt.plot(df.Date.iloc[i], df.Zscore_rolling.iloc[i], color='grey', linestyle='None', marker='o')
            day_count = 0
            
        elif abs(df.Zscore_rolling.iloc[i]) > 1 and abs(df.Zscore_rolling.iloc[i]) < 2.5 and position == False and (-1 <= df['Zscore_rolling'].iloc[i-1] <= 1): # Entry Rule
            plt.plot(df.Date.iloc[i], df.Zscore_rolling.iloc[i], color='green', linestyle='None', marker='^')
            position = True
            day_count = 0
        
        elif df.Zscore_rolling.iloc[i] * df.Zscore_rolling.iloc[i-1] < 0 and position == True: # Exit Rule
            plt.plot(df.Date.iloc[i], df.Zscore_rolling.iloc[i], color='orange', linestyle='None', marker='s')
            position = False
            day_count = 0
        
        
        
            
    
    plt.axhline(1, color = 'green')
    plt.axhline(-1, color = 'green')
    plt.axhline(2.5, color = 'red')
    plt.axhline(-2.5, color = 'red')
    plt.axhline(0, color = 'orange')
    plt.ylabel('Price')
    plt.xlabel('Date')
    plt.title("Zscore for pair {} and {}".format(df.columns[1],df.columns[2]))
    plt.show()
    

In [ ]:
graph_invest(df_test[1][0])

In [ ]:
def invest(df, entry_value, exit_value, stoploss_value, invest_value):
    vol_A = [0]
    vol_B = [0]
    total = [invest_value]
    ratio = list(df.Ratio)
    price_A = list(df.iloc[:,1])
    price_B = list(df.iloc[:,2])
    day_count = 0
    count = [0]
    position = False
    status = ['']
    
    for i in range(len(df)):
        if abs(df['Zscore_rolling'].iloc[i]) > stoploss_value:   # Stop-loss / Valeur absolue
            total.append(total[-1] + vol_A[-1] * price_A[i] + vol_B[-1] * price_B[i])
            vol_A.append(0)
            vol_B.append(0)
            day_count = 0
            count.append(day_count)
            position = False
            status.append("Stop Loss")
            
        elif df['Zscore_rolling'].iloc[i] * df['Zscore_rolling'].iloc[i-1]  < 0 and position == True:   # Clear Positions
            total.append(total[-1] + vol_A[-1] * price_A[i] + vol_B[-1] * price_B[i])
            vol_A.append(0)
            vol_B.append(0)
            day_count = 0
            count.append(day_count)
            position = False
            status.append("Clear")
            
        elif day_count > 50:    # Time-Limit
            total.append(total[-1] + vol_A[-1] * price_A[i] + vol_B[-1] * price_B[i])
            vol_A.append(0)
            vol_B.append(0)
            day_count = 0
            count.append(day_count)
            position = False
            status.append("Time Limit")
        
        elif df['Zscore_rolling'].iloc[i] > entry_value and (-1 <= df['Zscore_rolling'].iloc[i-1] <= 1) :    # Zscore above band
            if (position == False):
                vol_A.append(int(- (total[-1]/2) / price_A[i]))            # Sell A
                vol_B.append(int(+ (total[-1]/2) / price_B[i]))       # Buy B
                total.append(total[-1] + vol_A[-1] * price_A[i] + vol_B[-1] * price_B[i])
                position = True
                day_count = 0
                count.append(day_count)
                status.append("Uppper")
            else:
                vol_A.append(vol_A[-1])
                vol_B.append(vol_B[-1])
                total.append(total[-1])
                day_count += 1
                count.append(day_count)
                status.append("Unchanged")
            
        elif df['Zscore_rolling'].iloc[i] < - entry_value and (-1 <= df['Zscore_rolling'].iloc[i-1] <= 1):     # Zscore under band
            if (position == False):
                vol_A.append(int(+ (total[-1]/2) / price_A[i]))            # Buy A
                vol_B.append(int(- (total[-1]/2) / price_B[i]))       # Sell B
                total.append(total[-1] + vol_A[-1] * price_A[i] + vol_B[-1] * price_B[i])
                position = True
                day_count = 0
                count.append(day_count)
                status.append("Lower")
            else:
                vol_A.append(vol_A[-1])
                vol_B.append(vol_B[-1])
                total.append(total[-1])
                day_count += 1
                count.append(day_count)
                status.append("Unchanged")
        
        else:
            vol_A.append(vol_A[-1])
            vol_B.append(vol_B[-1])
            total.append(total[-1])
            day_count += 1
            count.append(day_count)
            status.append("Unchanged")

    df['Vol_A'] = vol_A[1:]
    df['Vol_B'] = vol_B[1:]
    df['Total'] = total[1:]
    df['Status'] = status[1:]
    df['Day_count'] = count[1:]
    
    return df

In [ ]:
df_test = [invest(df, 1, 0, 2.5, 100000) for df in df_test]

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(df_test[1])

In [ ]:
def display_profit(df_list, train):
    df_profit = pd.DataFrame(columns=['Name_A','Name_B','Profit'])
    index = 0
    for i in range(len(df_list)):
        df_profit = pd.concat([df_profit, pd.DataFrame(data = {'Name_A':df_list[i].columns[1], 'Name_B':df_list[i].columns[2], 'Profit':df_list[i].Total.iloc[-1]}, index = [index])])
        index += 1
    return df_profit

In [ ]:
df_final = display_profit(df_test, train)
df_final

In [ ]:
sum(df_final.Profit / len(df_final))

In [ ]:
addMonths(np.datetime64('2015-12-01'))

In [ ]:
def strat_by_pair(cache, combinations, train_start = np.datetime64('2014-01-01'), train_end = np.datetime64('2019-01-01')):
    test_final = []
    wallet = sep_tableau_train_test(combinations, train_start, train_end)[1]
    wallet = copy.deepcopy(wallet)
    test_start = train_end
    for i in range(len(wallet)):
        wallet[i]['Selected'] = False
    
    for i in range(1):
        # Séparation fenêtre glissante train/test (date de début et fin en paramètre) AVEC DES INDICES
        train,test = sep_tableau_train_test(combinations, train_start, train_end)    # Split Data between train and test with their date
        # Pair Selection (Cointegration + Stationarity)
        train = add_coint(train)
        train, ratio_score, log_score, same = pair_selection(train)
        train_list = [list(x.columns)[:3] for x in train]
        for j in range(len(wallet)):
            if list(wallet[j].columns[:3]) in train_list:
                wallet[j]['Selected'].loc[(wallet[j]['Date'] >= test_start) & (wallet[j]['Date'] < addMonths(test_start))] = True
                
                #print(wallet[j][(wallet[j]['Date'] >= test_start) & (wallet[j]['Date'] < addMonths(test_start))])
                
        # test[0]
        # w = (test[2]['Date'] >= np.datetime64('2019-01-02')) & (test[2]['Date'] < np.datetime64('2019-02-01'))

        # test[2]['Selected'] = w
        # test[2]
        # On lance l'investissement sur le test
        test = calcul_moyenne_mobile(copy.deepcopy(test))
        test_invest = [invest(df, 1, 0, 2.5, 100000) for df in test]
        test_final.append(display_profit(test_invest,train))
        # On incrémente d'un mois
        print(f"Pair n°{len(test_final)} Train date from {train_start} to {train_end}")
        
        # Incrémentation d'un mois de toutes les dates
        test_start = addMonths(test_start)
        train_start = addMonths(train_start)
        train_end = addMonths(train_end)
                          
    return test_final

In [ ]:
plot = strat_by_pair(cache, df_combi)

In [ ]:
plot[0],plot[1],plot[2]

# Others

In [ ]:
def linear_regression(df): #prends en entrée un dataframe représentant une paire
# Engle-Granger method (spread method)
    S1=df[df.columns[1]]
    S2=df[df.columns[2]]
    S1 = sm.add_constant(S1)
    results = sm.OLS(S2, S1).fit()
    S1 = S1[df.columns[1]]
    b = results.params[df.columns[1]]
    spread = S2 - b * S1
    df['Spread']=spread
    ''' Plot for seeing 
    spread.plot(figsize=(12,6))
    plt.axhline(spread.mean(), color='black')
    plt.legend(['Spread']);
    '''
    return df

In [ ]:
def rendement(df,j):
    #calculate the yield of the last j days
    rend1=[0 for i in range(j)]
    rend2=[0 for i in range(j)]
    for i in range(j,df.shape[0]):
        rend1.append((df.loc[i,df.columns[1]]-df.loc[i-j,df.columns[1]]) / df.loc[i-j,df.columns[1]])
        rend2.append((df.loc[i,df.columns[2]]-df.loc[i-j,df.columns[2]]) / df.loc[i-j,df.columns[2]])
    df["Rend_" + df.columns[1]]=rend1 
    df["Rend_" + df.columns[2]]=rend2 
    return df

In [ ]:
def volatility(df,j):
    vol1=[0 for i in range(j)]
    vol2=[0 for i in range(j)]
    for i in range(j,df.shape[0]):
        vol1.append(df[df.columns[1]][i-j:i].std())
        vol2.append(df[df.columns[2]][i-j:i].std())
    df["Vol_" + df.columns[1]]=vol1 
    df["Vol_" + df.columns[2]]=vol2 
    return df
    

In [ ]:
 find_stationarity(list_pairs) # All pair1-pair2 are not stationary (it makes sense)

In [ ]:
for i in range(len(list_pairs)):
    if len(list_pairs[i])!=0:
        for j in range(len(list_pairs[i])):
            list_pairs[i][j]=linear_regression(list_pairs[i][j])
            list_pairs[i][j]=zscore(list_pairs[i][j])
            if list_pairs[i][j] is False: #we verify that each zscore series is stationary
                print(list_pairs[i][j].columns[2]+" - "+ list_pairs[i][j].columns[1] + " not stationary")
            list_pairs[i][j]=volatility(list_pairs[i][j],30)
            

            
            
list_pairs[13][0]["Zscore"].plot(figsize=(12,6))
plt.axhline(list_pairs[13][0]["Zscore"].mean())
plt.axhline(list_pairs[13][0]["Zscore"].std(), color='red')
plt.axhline(-list_pairs[13][0]["Zscore"].std(), color='green')
plt.show()

In [ ]:
print(list_pairs[13][0])
print(adfuller(list_pairs[13][0]["Zscore"])[1])

In [ ]:
df_invest[0][0]

In [ ]:
def signal_invest(liste_paire):
    for i in range(len(liste_paire)):
        for j in range(len(liste_paire[i])):